In [13]:
import numpy as np

In [14]:
def mps_psi1_even(N):
    assert N % 2 == 0, "N muss gerade sein"
    A = np.zeros((2, 2, 2), dtype=complex)
    B = np.zeros((2, 2, 2), dtype=complex)
    
    A[0, 0, 0] = 1 / np.sqrt(2)
    A[1, 1, 0] = 1 / np.sqrt(2)
    B[0, 0, 1] = 1 / np.sqrt(2)
    B[1, 1, 1] = 1 / np.sqrt(2)
    
    mps = [A if i % 2 == 0 else B for i in range(N)]
    return mps

def mps_psi1_odd(N):
    assert N % 2 == 1, "N muss ungerade sein"
    A = np.zeros((2, 2, 2), dtype=complex)
    B = np.zeros((2, 2, 2), dtype=complex)
    
    A[0, 0, 0] = 1 / np.sqrt(2)
    A[1, 1, 0] = 1 / np.sqrt(2)
    B[0, 0, 1] = 1 / np.sqrt(2)
    B[1, 1, 1] = 1 / np.sqrt(2)
    
    mps = [A if i % 2 == 0 else B for i in range(N)]
    return mps

def mps_psi2(N):
    A = np.zeros((2, 2, 2), dtype=complex)
    A[0, 0, 0] = 1 / np.sqrt(2)
    A[1, 1, 0] = 1 / np.sqrt(2)
    A[0, 0, 1] = 1 / np.sqrt(2)
    A[1, 1, 1] = 1 / np.sqrt(2)
    
    mps = [A for _ in range(N)]
    return mps


# Exercise 2
Throughout this exercise we will use the following labelling of legs:
![title](labelling.png)

We will also be using the convention that we include trivial (i.e., one-dimensional) "phantom" singular values at either end of the MPS chain.

## 2.1

The first step is to implement functions that, given a MPS representation $M = [M^{[1]\sigma_1} , . . . ,M^{[N]\sigma_N}]$, compute the left-normalized
representation $[A^{[1]}, . . . ,A^{[n]}]$ and the right-normalized representation $[B^{[1]}, . . . ,B^{[n]}]$.

In [15]:
def left_normalize(Ms):
    """
    Convert a MPS to a left-normalized MPS.

    Parameters
    ----------
    Ms : list of rank 3 tensors.
        A MPS representation of the state.

    Returns
    -------
    As : list of rank 3 tensors.
        A left-normalized MPS representation of the state.
    """
    As = []
    for i in range(len(Ms)):
        M = Ms[i]
        d, D1, D2 = M.shape
        M = M.reshape(d * D1, D2)
        U, S, Vh = np.linalg.svd(M, full_matrices=False)
        U = U.reshape(d, D1, -1)
        As.append(U)
        if i < len(Ms) - 1:
            Ms[i + 1] = np.tensordot(np.diag(S), Vh, axes=(1, 0)).reshape(-1, *Ms[i + 1].shape[1:])
    return As


def right_normalize(Ms):
    """
    Convert a MPS to a right-normalized MPS.

    Parameters
    ----------
    Ms : list of rank 3 tensors.
        A MPS representation of the state.

    Returns
    -------
    Bs : list of rank 3 tensors.
        A right-normalized MPS representation of the state.
    """
    Bs = []
    for i in range(len(Ms) - 1, -1, -1):
        M = Ms[i]
        d, D1, D2 = M.shape
        M = M.reshape(d, D1 * D2)
        U, S, Vh = np.linalg.svd(M, full_matrices=False)
        Vh = Vh.reshape(-1, D1, D2)
        Bs.insert(0, Vh)
        if i > 0:
            Ms[i - 1] = np.tensordot(Ms[i - 1], np.diag(S), axes=(2, 0)).reshape(*Ms[i - 1].shape[:-1], -1)
    return Bs

## 2.2

Implement a function that constructs the Vidal canonical form and returns the matrices $\Gamma = [\Gamma^{[1]\sigma_1} , . . . ,\Gamma^{[N]\sigma_N}]$ and $\Lambda = [\Lambda^{[1]} , . . . ,\Lambda^{[N-1]}]$.

In [16]:
def truncate(U, S, Vh):
    """
    Remove singular values below threshold.
    """
    l = np.sum(np.abs(S) > 1e-8)
    return U[:, :l], S[:l], Vh[:l, :]

def vidal_form(Ms):
    """
    Convert a MPS to Vidal canonical form.

    Parameters
    ----------
    Ms : list of rank 3 tensors.
        A MPS representation of the state.

    Returns
    -------
    Lambdas : list of rank 1 tensors.
        The Schmidt values of every site.

    Gammas : list of rank 3 tensors.
        The Gamma matrices in the Vidal canonical form.
    """
    Lambdas = [np.array([1])]
    Gammas = []
    
    for i in range(len(Ms)):
        M = Ms[i]
        d, D1, D2 = M.shape
        M = M.reshape(d * D1, D2)
        U, S, Vh = np.linalg.svd(M, full_matrices=False)
        U, S, Vh = truncate(U, S, Vh)
        U = U.reshape(d, D1, -1)
        Gammas.append(U)
        Lambdas.append(S)
        if i < len(Ms) - 1:
            next_shape = Ms[i + 1].shape
            Ms[i + 1] = np.tensordot(np.diag(S), Vh, axes=(1, 0)).reshape(-1, *next_shape[1:])
    
    return Lambdas, Gammas

You can use the implemented functions to get the Vidal's canonical form for states $|\Psi_1>$ and $|\Psi_2>$. 

In [17]:
def phi1(N):
    """
    Get MPS form of phi1.

    Parameters
    ----------
    N : int
        The number of sites.

    Returns
    -------
    Lambdas
        The Schmidt values of phi1.

    Gammas
        The Gamma matrices in the Vidal canonical form of phi1.
    """
    if N % 2 == 0:
        Ms = mps_psi1_even(N)
    else:
        Ms = mps_psi1_odd(N)
    
    Lambdas, Gammas = vidal_form(Ms)
    return Lambdas, Gammas

def phi2(N):
    """
    Get MPS form of phi2.

    Parameters
    ----------
    N : int
        The number of sites.

    Returns
    -------
    Lambdas
        The Schmidt values of phi2.
        
    Gammas
        The Gamma matrices in the Vidal canonical form of phi2.
    """
    Ms = mps_psi2(N)
    Lambdas, Gammas = vidal_form(Ms)
    return Lambdas, Gammas

## 2.4

Write a function that, given two generic states in the Vidal's canonical form, evaluates their overlap as describe in the exercise sheet. Then, calculate the overlap between $|\Psi_1>$ and $|\Psi_2>$ for N = 30 spins.

In [18]:
def overlap(Lambdas1, Gammas1, Lambdas2, Gammas2):
    """
    Compute the overlap of two wave functions given in Vidal canonical form.

    Parameters
    ----------
    Lambdas1 : list of rank 1 tensors.
        Schmidt values of the first state.

    Gammas1 : list of rank 3 tensors.
        Gamma matrices of the first state.

    Lambdas2 : list of rank 1 tensors.
        Schmidt values of the second state.

    Gammas2 : list of rank 3 tensors.
        Gamma matrices of the second state.

    Returns
    -------
    complex
        Overlap of the two states
    """
    N = len(Gammas1)
    assert N == len(Gammas2), "Die Anzahl der Gamma-Matrizen muss gleich sein"
    
    # Initialisiere das Überlappungsprodukt
    overlap = 1.0 + 0j
    
    for i in range(N):
        # Kontrahiere die Gamma-Matrizen und Schmidt-Werte
        Gamma1 = Gammas1[i]
        Gamma2 = Gammas2[i]
        Lambda1 = Lambdas1[i]
        Lambda2 = Lambdas2[i]
        
        # Berechne das Produkt der Schmidt-Werte
        overlap *= np.dot(Lambda1, Lambda2)
        
        # Kontrahiere die Gamma-Matrizen
        overlap *= np.tensordot(np.conj(Gamma1), Gamma2, axes=([0, 1, 2], [0, 1, 2]))
    
    # Berücksichtige den letzten Schmidt-Wert
    overlap *= np.dot(Lambdas1[-1], Lambdas2[-1])
    
    return overlap

In [19]:
N = 30
Lambdas1, Gammas1 = phi1(N)
Lambdas2, Gammas2 = phi2(N)
print("Numeric:   ", overlap(Lambdas1, Gammas1, Lambdas2, Gammas2))
print("Analytical:", 2/np.sqrt(2)/(np.sqrt(2))**N)

ValueError: cannot reshape array of size 2 into shape (2,2)

Verify the correct implementation of your function by calculating the normalization of an arbitrary MPS state in canonical form (should be equal to 1).

In [ ]:
def random_MPS(d, N):
    """
    Returns a random MPS with bond dimension d.

    Parameters
    ----------
    d : int
        The bond dimension.

    N : int
        The number of sites.

    Returns
    -------
    Ms : list of rank 3 tensors.
        A random MPS representation of the state.
    """
    Ms = []
    for i in range(N):
        if i == 0 or i == N - 1:
            # First and last tensor have bond dimension 1
            M = np.random.rand(2, 1, d) + 1j * np.random.rand(2, 1, d)
        else:
            M = np.random.rand(2, d, d) + 1j * np.random.rand(2, d, d)
        Ms.append(M)
    return Ms

def check_left_normalisation(A):
    """
    Returns True if A is left-normalized.
    """
    E = np.tensordot(A, np.conjugate(A), axes=([0, 1], [0, 1]))
    return np.linalg.norm(E - np.eye(np.shape(E)[0])) < 1e-4

def check_right_normalisation(A):
    """
    Returns True if A is right-normalized.
    """
    E = np.tensordot(A, np.conjugate(A), axes=([0, 2], [0, 2]))
    return np.linalg.norm(E - np.eye(np.shape(E)[0])) < 1e-4

def check_properties(Lambdas, Gammas):
    properties = True
    for i in range(len(Gammas)):
        A = np.tensordot(np.diag(Lambdas[i]), Gammas[i], axes=(1, 1))
        A = np.transpose(A, [1, 0, 2])
        B = np.tensordot(Gammas[i], np.diag(Lambdas[i+1]), axes=(2, 0))
        properties = properties and check_left_normalisation(A) and check_right_normalisation(B)
    return properties

# Beispielaufruf zur Überprüfung der Normierung und kanonischen Form
Lambdas_rand, Gammas_rand = vidal_form(random_MPS(2, 30))
print("Norm:", overlap(Lambdas_rand, Gammas_rand, Lambdas_rand, Gammas_rand))
print("Is canonical:", check_properties(Lambdas_rand, Gammas_rand))

Norm: (7.226848875514486e+17+0j)


ValueError: shape-mismatch for sum